In [2]:
import pandas as pd

In [2]:
file_txt = open('wr112006.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)

file_csv.close()
file_txt.close()

'''Создаем файл в формате csv'''

'Создаем файл в формате csv'

In [3]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)

'''Создаем датафрейм'''


'Создаем датафрейм'

In [4]:
df_filtered = df[(df['month'] > 4) & (df['month'] < 10)]

'''Оставляем только нужные месяцы: май, июнь, июль, август, сентябрь'''

print(df_filtered)

         station  year  month  day  summary  min_temperature  \
120        20046  1990      5    1        0            -17.9   
121        20046  1990      5    2        0            -17.5   
122        20046  1990      5    3        0            -18.1   
123        20046  1990      5    4        0            -13.7   
124        20046  1990      5    5        0            -16.0   
...          ...   ...    ...  ...      ...              ...   
5770901    37663  2020      9   26        0             10.0   
5770902    37663  2020      9   27        0             11.6   
5770903    37663  2020      9   28        0             11.3   
5770904    37663  2020      9   29        0              9.1   
5770905    37663  2020      9   30        0              5.3   

         avarage_temperature  max_temperature  precipitation  
120                    -16.7            -12.5            0.0  
121                    -15.3            -12.9            0.0  
122                    -14.9            -1

In [5]:
df_cleaned = df_filtered.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_cleaned.dropna(inplace=True)
df_cleaned['station'] = df_cleaned['station'].astype(int)

print(df_cleaned)

df_cleaned.to_csv('cleaned_dataset.csv',  index=False)

'''Удаляем ненужные столбы и строки с пустями значениями, записываем в новый файл '''

         station  year  month  day  avarage_temperature  max_temperature
120        20046  1990      5    1                -16.7            -12.5
121        20046  1990      5    2                -15.3            -12.9
122        20046  1990      5    3                -14.9            -12.6
123        20046  1990      5    4                -12.6             -9.5
124        20046  1990      5    5                -13.2            -11.0
...          ...   ...    ...  ...                  ...              ...
5770901    37663  2020      9   26                 14.7             21.1
5770902    37663  2020      9   27                 13.8             16.6
5770903    37663  2020      9   28                 12.8             15.7
5770904    37663  2020      9   29                 13.1             19.4
5770905    37663  2020      9   30                 10.1             17.3

[2376728 rows x 6 columns]


'Удаляем ненужные столбы и строки с пустями значениями, записываем в новый файл '

In [39]:
df_first = pd.read_csv('cleaned_dataset.csv')
df_second = pd.read_csv('stations.csv')

         station  year  month  day  avarage_temperature  max_temperature
0          20046  1990      5    1                -16.7            -12.5
1          20046  1990      5    2                -15.3            -12.9
2          20046  1990      5    3                -14.9            -12.6
3          20046  1990      5    4                -12.6             -9.5
4          20046  1990      5    5                -13.2            -11.0
...          ...   ...    ...  ...                  ...              ...
2376723    37663  2020      9   26                 14.7             21.1
2376724    37663  2020      9   27                 13.8             16.6
2376725    37663  2020      9   28                 12.8             15.7
2376726    37663  2020      9   29                 13.1             19.4
2376727    37663  2020      9   30                 10.1             17.3

[2376728 rows x 6 columns]


In [42]:
df_new = df_first.merge(df_second, how='left', on='station')

In [43]:
df_new['region'].value_counts()

Иркутская область                      105834
Республика Бурятия                      70966
Красноярский край                       55508
Саратовская область                     37942
Томская область                         37913
Алтайский край                          37424
Забайкальский край                      35573
Краснодарский край                      32918
Республика Дагестан                     32389
Ростовская область                      29742
Новосибирская область                   23715
Республика Алтай                        23684
Республика Тыва                         23337
Ставропольский край                     19919
Оренбургская область                    18972
Волгоградская область                   18972
Кемеровская область - Кузбасс           18972
Республика Саха (Якутия)                18924
Белгородская область                    14229
Курская область                         14229
Республика Хакасия                      14229
Воронежская область               

In [34]:
df_new.to_csv('full_dataset.csv',  index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   station           530 non-null    int64  
 1   name of station   530 non-null    object 
 2   number of region  175 non-null    float64
 3   region            175 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 16.7+ KB
